# Management - Gen 2 Opt

In [1]:
import qcportal as ptl
import pandas as pd
import datetime
import time

from management import get_failed_optimizations

In [2]:
# connect without auth
# read only
client = ptl.FractalClient()

In [2]:
# connect with authentication, therefore write access
# don't use unless you plan to submit things
client = ptl.FractalClient.from_file()

In [3]:
client

FractalClient(server_name='The MolSSI QCArchive Server', address='https://api.qcarchive.molssi.org:443/', username='ddotson')

## Dataset tracking - OpenFF Gen 2 Opt Set 4 eMolecules Discrepancy

In [31]:
dataset = "OpenFF Gen 2 Opt Set 4 eMolecules Discrepancy"

In [32]:
ds = client.get_collection("OptimizationDataset", dataset)

In [33]:
ds.list_specifications()

,Description
Name,
default,Standard OpenFF optimization quantum chemistry...


In [35]:
ds.status("default")

,default
COMPLETE,2196
ERROR,5


In [36]:
res = get_failed_optimizations(ds, 'default', client)

In [38]:
for i in sorted([i.id for i in res if i.status != 'COMPLETE']):
    print(i)

18436178
18437465
18437470
18438079
18438248


In [46]:
errors = (i.get_error().error_message for i in res if i.status == 'ERROR')

In [51]:
print(next(errors))

geomeTRIC run_json error:
Traceback (most recent call last):
  File "/opt/conda/envs/qcfractal/lib/python3.6/site-packages/geometric/run_json.py", line 214, in geometric_run_json
    geometric.optimize.Optimize(coords, M, IC, engine, None, params)
  File "/opt/conda/envs/qcfractal/lib/python3.6/site-packages/geometric/optimize.py", line 1331, in Optimize
    return optimizer.optimizeGeometry()
  File "/opt/conda/envs/qcfractal/lib/python3.6/site-packages/geometric/optimize.py", line 1301, in optimizeGeometry
    raise GeomOptNotConvergedError("Optimizer.optimizeGeometry() failed to converge.")
geometric.errors.GeomOptNotConvergedError: Optimizer.optimizeGeometry() failed to converge.



For convergence errors, if we can look at the initial and final molecules in NGLview, can give indications as to why:

In [57]:
opt = res[0]

In [58]:
opt.get_initial_molecule()

_ColormakerRegistry()

NGLWidget()

In [60]:
opt.get_final_molecule()

NGLWidget()

All of these are convergence failures.

## Dataset tracking - OpenFF Gen 2 Opt Set 2 Coverage

In [53]:
dataset = "OpenFF Gen 2 Opt Set 2 Coverage"

In [54]:
ds = client.get_collection("OptimizationDataset", dataset)

In [55]:
ds.list_specifications()

,Description
Name,
default,Standard OpenFF optimization quantum chemistry...


In [57]:
ds.status("default")

,default
COMPLETE,359
INCOMPLETE,10
ERROR,4


In [58]:
res = get_failed_optimizations(ds, 'default', client)

In [59]:
[i for i in res if i.status != 'COMPLETE']

[OptimizationRecord(id='18433828', status='ERROR'),
 OptimizationRecord(id='18433929', status='INCOMPLETE'),
 OptimizationRecord(id='18433930', status='INCOMPLETE'),
 OptimizationRecord(id='18433937', status='INCOMPLETE'),
 OptimizationRecord(id='18433944', status='INCOMPLETE'),
 OptimizationRecord(id='18433984', status='INCOMPLETE'),
 OptimizationRecord(id='18433988', status='INCOMPLETE'),
 OptimizationRecord(id='18434010', status='INCOMPLETE'),
 OptimizationRecord(id='18434013', status='INCOMPLETE'),
 OptimizationRecord(id='18434015', status='INCOMPLETE'),
 OptimizationRecord(id='18434075', status='INCOMPLETE'),
 OptimizationRecord(id='18434085', status='ERROR'),
 OptimizationRecord(id='18434088', status='ERROR'),
 OptimizationRecord(id='18434121', status='ERROR')]

In [80]:
errors = ((i.id, i.get_error().error_message) for i in res if i.status == 'ERROR')

In [81]:
for i in errors:
    print(i[0])

18433828
18434085
18434088
18434121


In [79]:
print(next(errors)[1])

geomeTRIC run_json error:
Traceback (most recent call last):
  File "/home/bpp4/miniconda3/envs/qca_manager/lib/python3.7/site-packages/geometric/run_json.py", line 214, in geometric_run_json
    geometric.optimize.Optimize(coords, M, IC, engine, None, params)
  File "/home/bpp4/miniconda3/envs/qca_manager/lib/python3.7/site-packages/geometric/optimize.py", line 1331, in Optimize
    return optimizer.optimizeGeometry()
  File "/home/bpp4/miniconda3/envs/qca_manager/lib/python3.7/site-packages/geometric/optimize.py", line 1298, in optimizeGeometry
    self.calcEnergyForce()
  File "/home/bpp4/miniconda3/envs/qca_manager/lib/python3.7/site-packages/geometric/optimize.py", line 1002, in calcEnergyForce
    spcalc = self.engine.calc(self.X, self.dirname)
  File "/home/bpp4/miniconda3/envs/qca_manager/lib/python3.7/site-packages/geometric/engine.py", line 873, in calc
    return self.calc_new(coords, dirname)
  File "/home/bpp4/miniconda3/envs/qca_manager/lib/python3.7/site-packages/geometr

Some convergence errors.
Mostly unknown errors.

Let's try re-running the uknown errors.

In [20]:
resubmit = [i for i in res if i.status == 'ERROR' and "Could not converge" not in i.get_error().error_message]

In [52]:
for i in resubmit:
    print(i.id)

18433929
18433930
18433937
18433944
18433984
18433988
18434010
18434013
18434015
18434075


In [23]:
len(resubmit)

10

In [25]:
for i in resubmit:
    client.modify_tasks(operation='restart', base_result=i.id)

In [26]:
ds = client.get_collection("OptimizationDataset", dataset)

In [27]:
res = get_failed_optimizations(ds, 'default', client)

In [29]:
len([i for i in res if i.status == 'ERROR'])

4

In [30]:
len([i for i in res if i.status == 'INCOMPLETE'])

10